In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, h1ere's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ue20cs344-nam-assignment-2/edgelist.csv
/kaggle/input/ue20cs344-nam-assignment-2/sample_submission.csv
/kaggle/input/ue20cs344-nam-assignment-2/node_data.csv
/kaggle/input/ue20cs344-nam-assignment-2/features.pt


In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
el = pd.read_csv('/kaggle/input/ue20cs344-nam-assignment-2/edgelist.csv')
nd = pd.read_csv('/kaggle/input/ue20cs344-nam-assignment-2/node_data.csv')

In [4]:
nd.head()

,id,branch,train_mask,test_mask
0,0,ECE,True,False
1,1,ECE,True,False
2,2,CSE,True,False
3,3,EEE,True,False
4,4,NaN,False,True


In [5]:
el.head()

,source,target
0,0,1378
1,0,1544
2,0,6092
3,0,7636
4,0,14442


In [6]:
G = nx.Graph()

In [7]:
#nd.set_index('id',inplace=True)
for node_id,node_attrs in nd.iterrows():
    attrs = node_attrs.to_dict()
    G.add_node(id,**attrs)

for id,row in el.iterrows():
    source = row['source']
    target = row['target']
    G.add_edge(source,target)
    
#nx.draw(G,with_labels=True)

In [8]:
print(format(G.number_of_edges()))
print(format(G.number_of_nodes()))

44324
19718


In [9]:
#Install the necessary packages
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install --upgrade torch-cluster

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for torch-cluster: filename=torch_cluster-1.6.1-cp37-cp37m-linux_x86_64.whl size=739190 sha256=4da9f0e00a3b6bb007e54c89db28cecc6ddd9c5d99ce72a8ac3f9a3c18864f1f
  Stored in directory: /root/.cache/pip/wheels/59/a4/3a/bd84e20d689ff3f2478a7bb718c3a5855710d5c77a054df5fb
Successfully built torch-cluster


In [10]:
#Making all the necessary imports 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import networkx as nx
import os.path as osp
import torch  
from tqdm.notebook import tqdm 
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import Node2Vec
import torch_cluster
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [11]:
pip install node2vec

  Preparing metadata (setup.py) ... - done
  Created wheel for node2vec: filename=node2vec-0.4.3-py3-none-any.whl size=5980 sha256=b2ba0b4c4a413b90c3d95ab75a6e4da180559471e8a3ef15f8c154b5a67d1ed8
  Stored in directory: /root/.cache/pip/wheels/41/d8/5d/28571981b2dc5a7003e03031a9f42c9a68f323b8fb1ddbb341
Successfully built node2vec
Note: you may need to restart the kernel to use updated packages.


In [12]:
from node2vec import Node2Vec
node2vec = Node2Vec(G, dimensions=128 , walk_length=80 , num_walks=10 , p=1 , q=0.5)
model = node2vec.fit(window=10, min_count=1)

Computing transition probabilities:   0%|          | 0/19718 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [10:04<00:00, 60.50s/it]


In [13]:
features=torch.load('/kaggle/input/ue20cs344-nam-assignment-2/features.pt')

In [14]:
from sklearn.preprocessing import LabelEncoder
labels = LabelEncoder()
nd['branch'] = labels.fit_transform(nd['branch'])
nd.head()

,id,branch,train_mask,test_mask
0,0,1,True,False
1,1,1,True,False
2,2,0,True,False
3,3,2,True,False
4,4,3,False,True


In [15]:
from torch_geometric.data import Data
edge_index = torch.tensor(list(G.edges)).t().contiguous()
x = features
y = torch.tensor(nd['branch'].values,)
data = Data(x=x, edge_index=edge_index , y=y)
data.train_mask = nd["train_mask"].values
data.test_mask = nd["test_mask"].values

In [16]:
#graphsage
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [17]:
model = GraphSAGE(data.num_features, 16, 3)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [18]:
## Train the model
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    print(f"loss:{loss}, epoch:{epoch}")
    loss.backward()
    optimizer.step()

loss:1.078887939453125, epoch:0
loss:1.0700348615646362, epoch:1
loss:1.0617119073867798, epoch:2
loss:1.0520209074020386, epoch:3
loss:1.0410798788070679, epoch:4
loss:1.030768632888794, epoch:5
loss:1.0188578367233276, epoch:6
loss:1.0034524202346802, epoch:7
loss:0.9924741387367249, epoch:8
loss:0.9796929359436035, epoch:9
loss:0.962448000907898, epoch:10
loss:0.9433087706565857, epoch:11
loss:0.9297482371330261, epoch:12
loss:0.9097240567207336, epoch:13
loss:0.8954039216041565, epoch:14
loss:0.8737378716468811, epoch:15
loss:0.856985330581665, epoch:16
loss:0.8411339521408081, epoch:17
loss:0.819584310054779, epoch:18
loss:0.8074754476547241, epoch:19
loss:0.7773948907852173, epoch:20
loss:0.7644479870796204, epoch:21
loss:0.7493978142738342, epoch:22
loss:0.7286868691444397, epoch:23
loss:0.7143148183822632, epoch:24
loss:0.7008190751075745, epoch:25
loss:0.6901144981384277, epoch:26
loss:0.688529908657074, epoch:27
loss:0.6682896614074707, epoch:28
loss:0.6579039692878723, epoch

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.eval()
_, pred = model(data.x.to(device), data.edge_index.to(device)).max(dim=1)
test_preds = pred[data.test_mask].detach().cpu().numpy()
test_ids = nd['id'][data.test_mask].values

In [20]:
# Evaluate the model
model.eval()
out = model(data.x, data.edge_index)
pred = out.argmax(dim=1)
train_acc = pred[data.train_mask].eq(data.y[data.train_mask]).sum().item() / data.train_mask.sum().item()
test_acc = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()
print(f'GraphSage Train Accuracy: {train_acc:.4f}')
print(f'GraphSage Test Accuracy: {test_acc:.4f}')

GraphSage Train Accuracy: 0.8899
GraphSage Test Accuracy: 0.0000


In [21]:
test_preds = labels.inverse_transform(test_preds)
print(test_preds)

['ECE' 'EEE' 'ECE' ... 'EEE' 'EEE' 'CSE']


In [22]:
submission_df = pd.DataFrame({'id': test_ids, 'branch': test_preds})
submission_df.to_csv('submission.csv', index=False)
len(submission_df)

3944